<a href="https://colab.research.google.com/github/wwookk/Code-AI/blob/main/CodeBERT_1_N(MSR%2BDIV%2BJUL)_nofunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cwe file/MSR+julite+Div_no_func.csv')

In [ ]:
df.loc[df['vul'] == 0, 'CWE ID'] = 'safe'

In [ ]:
cwe_to_label = {
    'safe': 0,
    'CWE-119': 1,
    'CWE-20': 2,
    'CWE-125': 3,
    'CWE-787': 4,
    'CWE-415': 5,
    'CWE-399': 6,
    'CWE-416': 7,
    'CWE-476': 8,
    'CWE-190': 9
}
df['labels'] = df['CWE ID'].map(cwe_to_label)

In [ ]:
df

,Unnamed: 0,CWE ID,code,vul,lang,labels
0,0,safe,isdn_net_hangup(struct net_device *d){ isdn_ne...,0,c,0
1,1,safe,isdn_net_ciscohdlck_slarp_send_request(isdn_ne...,0,c,0
2,2,safe,static __inline__ void isdn_net_device_stop_qu...,0,c,0
3,3,safe,"isdn_net_new(char *name, struct net_device *ma...",0,c,0
4,4,safe,isdn_net_rm(char *name){ u_long flags; isdn_ne...,0,c,0
...,...,...,...,...,...,...
16390,16390,CWE-415,void func()\n{\n wchar_t * data;\n \n ...,1,cpp,5
16391,16391,CWE-415,namespace func\n{\nfunc::func(wchar_t * dataCo...,1,cpp,5
16392,16392,CWE-415,namespace func\n{\nfunc::func(wchar_t * dataCo...,1,cpp,5
16393,16393,CWE-415,class func \n{\n public:\n func(cons...,1,cpp,5


In [ ]:
df.labels.value_counts()

labels
0    3257
1    2800
2    1978
3    1681
4    1536
5    1380
6    1059
7    1057
8     912
9     735
Name: count, dtype: int64

In [ ]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base-mlm", num_labels = 10)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base-mlm")
batch_size = 16
epoch_num = 1000
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    df['code'] = df['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    # 매크로 정의 제거
    df['code'] = df['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 전처리 지시문 제거
    df['code'] = df['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#endif'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    df['code'] = df['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    df['code'] = df['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return;'), "", regex=True)
    # 데이터프레임을 CSV 파일로 저장
    df.to_csv(file_name, index=False)
    return df  # 수정된 데이터프레임 반환

def tokenized(examples):
  return tokenizer(examples['code'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['code'],batched=True)

In [ ]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics

In [ ]:
from transformers import TrainerCallback
class CustomTrainerCallback(TrainerCallback):
    def __init__(self, trainer, eval_dataset, early_stopping_patience=5):
        self.trainer = trainer
        self.eval_dataset = eval_dataset
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = float('inf')
        self.patience_counter = 0
        self.incorrect_samples = []

    def on_evaluate(self, args, state, control, **kwargs):
        eval_loss = kwargs.get('metrics', {}).get('eval_loss')

        if eval_loss is not None:
            if eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1

            if self.patience_counter >= self.early_stopping_patience:
                print(f"No improvement in {self.early_stopping_patience} evaluation steps. Stopping training early.")
                control.should_training_stop = True

        # Evaluate를 위한 DataLoader 설정 및 평가 로직
        eval_dataloader = self.trainer.get_eval_dataloader(self.eval_dataset)
        self.trainer.model.eval()
        for batch in eval_dataloader:
            batch = {k: v.to(self.trainer.args.device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = self.trainer.model(**batch)
                predictions = torch.argmax(outputs.logits, dim=-1)
                labels = batch['labels']
                incorrect_mask = predictions != labels

            if incorrect_mask.any():
                incorrect_samples = {k: v[incorrect_mask].cpu().numpy() for k, v in batch.items()}
                self.incorrect_samples.append(incorrect_samples)

        print("Collected incorrect predictions for further analysis.")

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1000,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)
callback = CustomTrainerCallback(encoded_dataset['test'], 5)

# 트레이너 초기화 및 사용자 정의 콜백 추가

# 콜백 인스턴스 생성 및 트레이너 초기화

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics
)
callback = CustomTrainerCallback(trainer, encoded_dataset['test'], 5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    callbacks=[callback]
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9
500,1.833400,1.415692,0.516772,0.516772,0.494723,0.441874,0.516772,0.738116,0.478791,0.400381,0.407970,0.433697,0.905852,0.304196,0.276347,0.140056,0.333333
1000,1.184600,1.077010,0.654808,0.654808,0.646200,0.610905,0.654808,0.843340,0.602965,0.557252,0.598465,0.602299,0.918851,0.568306,0.496195,0.348485,0.572890
1500,0.842900,1.000340,0.688758,0.688758,0.688326,0.658327,0.688758,0.880543,0.643023,0.607516,0.596352,0.618893,0.938053,0.649457,0.600619,0.447115,0.601695
2000,0.592300,1.016322,0.714170,0.714170,0.713834,0.689591,0.714170,0.867764,0.663534,0.655172,0.676171,0.636161,0.930572,0.674457,0.646048,0.510067,0.635965
2500,0.426900,1.112526,0.716609,0.716609,0.718030,0.692688,0.716609,0.883887,0.667969,0.652549,0.672978,0.632997,0.938826,0.696817,0.610801,0.518167,0.651885
3000,0.349000,1.134272,0.734092,0.734092,0.733682,0.710713,0.734092,0.877246,0.686810,0.700000,0.694476,0.656983,0.906287,0.695055,0.662519,0.576490,0.651261
3500,0.285400,1.157189,0.745477,0.745477,0.744972,0.720278,0.745477,0.891382,0.717681,0.710327,0.689204,0.637710,0.927536,0.724684,0.654485,0.576000,0.673774
4000,0.241700,1.133740,0.752592,0.752592,0.753684,0.734143,0.752592,0.883103,0.711811,0.718323,0.717435,0.662407,0.945679,0.697013,0.690722,0.609901,0.705036


Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
No improvement in 5 evaluation steps. Stopping training early.
Collected incorrect predictions for further analysis.


TrainOutput(global_step=4000, training_loss=0.7195331439971924, metrics={'train_runtime': 5464.5089, 'train_samples_per_second': 2100.097, 'train_steps_per_second': 105.041, 'total_flos': 2.1044081225416704e+16, 'train_loss': 0.7195331439971924, 'epoch': 6.968641114982578})

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

def plot_training_loss(log_dir):
    event_files = [os.path.join(log_dir, f) for f in os.listdir(log_dir) if 'events' in f]
    if not event_files:
        raise ValueError("No event files found in log directory.")

    for event_file in event_files:
        losses = {'Step': [], 'Training Loss': [], 'Validation Loss': []}
        for e in tf.compat.v1.train.summary_iterator(event_file):
            for v in e.summary.value:
                if v.tag == 'train/loss':
                    losses['Training Loss'].append(v.simple_value)
                    losses['Step'].append(e.step)
                elif v.tag == 'eval/loss':
                    losses['Validation Loss'].append(v.simple_value)

        if losses['Step']:
            df = pd.DataFrame(losses).set_index('Step')
            plt.figure(figsize=(10, 5))
            plt.plot(df['Training Loss'], label='Training Loss')
            plt.plot(df['Validation Loss'], label='Validation Loss')
            plt.xlabel('Steps')
            plt.ylabel('Loss')
            plt.title('Training and Validation Loss')
            plt.legend()
            plt.grid(True)
            plt.show()

# 로그 디렉토리를 지정하여 그래프 그리기
plot_training_loss('./logs')

In [ ]:
trainer.save_model('/content/drive/MyDrive/codebert(nofunc)')